In [1]:
from hello import hello, hello3
from officelib.xllib import *
from officelib import const
import requests

HERR = (hello.HelloError, hello3.HelloError)
NLI = (hello.HelloError, hello3.NotLoggedInError)

from time import sleep
def set_mg(sp):
    call(app.setmg, 1, sp)        
        
def call(f, *args):
    
    while True:
        try:
            rv = f(*args)
        except NLI:
            app.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv
        
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on(v):
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on(v):
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    app.setmg(1, v)
    input("Purging line of air. Press enter when done.")
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name](mfcm)
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(2, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect(ip=None, v=None):
    global app
    v = v or input("Version: ").lower()
    ip = ip or input("Enter IP Address: ")
    if str(v)[0] == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [2]:
from scripts import wlan
wlan.ensure_wifi('pbstech')

Already on pbstech


In [17]:
init()
connect('169.254.110.21', 3)
all_off()

In [19]:
run("air", 0.5)

Testing  air MFC: 0.5 LPM Max
0.05 LPM:
0.10 LPM:
0.15 LPM:
0.20 LPM:
0.25 LPM:
0.30 LPM:
0.35 LPM:
0.40 LPM:418.7
0.45 LPM:470.5
0.50 LPM:516.9


In [20]:
run("n2", 0.5)

Testing  n2 MFC: 0.5 LPM Max
0.05 LPM:56.5
0.10 LPM:106.5
0.15 LPM:158.7
0.20 LPM:210.1
0.25 LPM:262.5
0.30 LPM:315.2
0.35 LPM:366.8
0.40 LPM:418.3
0.45 LPM:470.2
0.50 LPM:517.2


In [22]:
run("o2", .5)

Testing  o2 MFC: 0.5 LPM Max
0.05 LPM:56.8
0.10 LPM:107.1
0.15 LPM:158.5
0.20 LPM:210
0.25 LPM:261.5
0.30 LPM:313.7
0.35 LPM:365.4
0.40 LPM:416.2
0.45 LPM:467.2
0.50 LPM:516.8


In [21]:
run("co2", .1)

Purging line of air. Press enter when done.
Testing  co2 MFC: 0.1 LPM Max
0.01 LPM:30.1
0.02 LPM:
0.03 LPM:30.1
0.04 LPM:40.6
0.05 LPM:51.1
0.06 LPM:61.9
0.07 LPM:72.2
0.08 LPM:82.4
0.09 LPM:92.8
0.10 LPM:102.8
